In [ ]:
from collections import namedtuple
import pathlib
import random

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [ ]:
from orchid import (project_loader as opl)

In [ ]:
import toolz.curried as toolz

In [ ]:
# noinspection PyUnresolvedReferences
import UnitsNet

In [ ]:
project_filenames = {
    'bakken': 'frankNstein_Bakken_UTM13_FEET.ifrac',
    'montney': 'Project-frankNstein_Montney_UTM13_METERS.ifrac',
}

In [ ]:
test_data_path = pathlib.Path('c:/src/Orchid.IntegrationTestData/')
project_path_names = toolz.valmap(lambda fn: test_data_path.joinpath(fn), project_filenames)
project_path_names

In [ ]:
projects = toolz.valmap(
    lambda pn: opl.ProjectLoader(str(pn)).native_project(),
    project_path_names)
projects

In [ ]:
project_units = toolz.valmap(lambda p: p.ProjectUnits, projects)
project_units

In [ ]:
def collect_time_series(so_far, ts):
    return toolz.merge(so_far, {ts.DisplayName: ts})

def well_time_series(project):
    return toolz.reduce(collect_time_series, project.WellTimeSeriesList.Items, {})

project_time_series = toolz.valmap(well_time_series, projects)
project_time_series

In [ ]:
toolz.valmap(len, project_time_series)

In [ ]:
TimeSeriesSummary = namedtuple('TimeSeriesSummary', ['display_name', 'name',
                                                     'phenomenon', 'net_unit', 'suffix',
                                                     'start_time', 'stop_time'])


In [ ]:
def summarize_time_series(ts):
    time_series_range = ts.GetTimeSeriesRange()
    start = time_series_range.Item1
    stop = time_series_range.Item2
    return TimeSeriesSummary(ts.DisplayName, ts.Name,
                             ts.SampledQuantityName, ts.SampledQuantityType, ts.Suffix,
                             start, stop)

project_time_series_summaries = toolz.valmap(
    toolz.valmap(summarize_time_series),
    project_time_series)
project_time_series_summaries

In [ ]:
project_time_series_samples = toolz.valmap(
    toolz.valmap(toolz.compose(# list,
                               lambda ts: ts.GetOrderedTimeSeriesHistory())),
    project_time_series)
# samples_to_print = toolz.valmap(
#     toolz.map(len),
#     project_time_series_samples)
# samples_to_print

In [ ]:
first_well_time_series = projects['bakken'].WellTimeSeriesList.Items[0]
first_well_time_series

In [ ]:
first_well_time_series_samples = first_well_time_series.GetOrderedTimeSeriesHistory()
len(first_well_time_series_samples)
[first_well_time_series_samples[n].Value for n in range(5)]

In [ ]:
def generate_sample_indices(ts):
    candidate_indices = list(range(len(first_well_time_series_samples)))
    random.shuffle(candidate_indices)
    inner_indices = sorted(candidate_indices[:6])
    result = [0] + inner_indices + [len(first_well_time_series_samples) - 1]
    return result

In [ ]:
sample_indices = generate_sample_indices(first_well_time_series_samples)
sample_indices

In [ ]:
@toolz.curry
def make_sample_pressure(summary, sample):
    return UnitsNet.Pressure.From(UnitsNet.QuantityValue.op_Implicit(sample.Value), 
                                  project_units['bakken'].PressureUnit)

def sample_time_series(indices, summary, ts):
    result = toolz.pipe(indices,
                        # toolz.map(toolz.do(lambda n: print(n))),
                        toolz.map(lambda n: toolz.nth(n, ts)),
                        toolz.map(toolz.do(lambda s: print(s.Value))),
                        toolz.map(make_sample_pressure(summary)),
                        toolz.map(str),
                        list,
                        )
    return result
    
sample_time_series(sample_indices, time_series_summary, first_well_time_series_samples)